In [1]:
import os
import sqlite3
import pandas as pd


In [2]:
#reading raw data
#reading from folder data
rawdb_path = '../data/raw_realState_data.db'
conn = sqlite3.connect(rawdb_path)
query = 'SELECT * FROM properties'

df = pd.read_sql(query, conn)

conn.close()

In [3]:
#Datos duplicados, en su mayoría son propiedades diferentes construidas
#ya sea en condominios o en urbanizaciones cerradas. Estas son de construcción idéntica.
#Se consideraran todas estas propiedades.
print(df.shape)
print(df.drop_duplicates().shape)

(1522, 14)
(683, 14)


In [4]:
#columna titulo tiene 3 campos, separar en 3
df_1 = df.copy()
df_1[['tipo_de_propiedad', 'ofertado_como', 'ciudad']] = df_1['titulo'].str.split(' - ', expand=True)
df_1.drop('titulo', axis=1, inplace=True)
#df_1.head()


In [5]:
#columna ubicación, extraer la zona
df_2 = df_1.copy()
df_2['ubicacion_descr_lst'] = df_2['ubicacion_descr'].str.split(' - ', expand=False)
df_2['zona'] = df_2['ubicacion_descr_lst'].apply(lambda x : x[-2] if len(x) > 1 else None)
df_2.drop(['ubicacion_descr', 'ubicacion_descr_lst'], axis=1, inplace=True)
#df_2.head(1)


In [6]:
#separar la columna ciudad en sus tres componentes
df_3 = df_2.copy()
df_3[['ciudad', 'provincia', 'dpto']] = df_3['ciudad'].str.split(',', expand=True)
#df_3.head(1)

In [7]:
#extraer el año de fecha/mes constr y unirla con año
df_4 = df_3.copy()
df_4[['año_constr_1', 'mes']] = df_4['fecha_mes_constr'].str.split('/', expand=True)
df_4['año_constr'] = df_4['año_constr'].fillna(df_4['año_constr_1']).astype(int)
df_4.drop(['año_constr_1', 'mes', 'fecha_mes_constr'], axis=1, inplace=True)
df_4.head()

,descripcion,precio,ambientes,no_baños,terreno_m2,año_constr,no_dormitorios,area_constr_m2,estacionamientos,latitud,longitud,tipo_de_propiedad,ofertado_como,ciudad,zona,provincia,dpto
0,HERMOSA CASA EN VENTA URB LAS PALMERAS,67000.0,8.0,2.0,320.0,2017,3.0,140.0,NaN,-17.768509,-63.066427,Casa,En Venta,Santa Cruz de la Sierra,Este,Andrés Ibáñez,Santa Cruz
1,DEPARTAMENTO A ESTRENAR TORRE BARCELONA,67000.0,5.0,2.0,91.0,2018,2.0,91.0,NaN,-17.804298,-63.156515,Departamento,En Venta,Santa Cruz de la Sierra,Sur,Andrés Ibáñez,Santa Cruz
2,CASA EN VENTA ZONA SUR OESTE,68000.0,NaN,2.0,193.0,2015,3.0,87.0,NaN,-17.836460,-63.244579,Casa,En Venta,Santa Cruz de la Sierra,Suroeste,Andrés Ibáñez,Santa Cruz
3,"Depto en Condominio ITAIPU, ZONA EQUIPETROL",68000.0,2.0,1.0,None,2016,1.0,62.83,NaN,-17.755193,-63.195968,Departamento,En Venta,Santa Cruz de la Sierra,Equipetrol/NorOeste,Andrés Ibáñez,Santa Cruz
4,CASA EN ZONA DE CRECIMIENTO EN VENTA,68000.0,8.0,2.0,300.0,2022,3.0,110.0,NaN,-17.653397,-63.173199,Casa,En Venta,Santa Cruz de la Sierra,Norte,Andrés Ibáñez,Santa Cruz


In [8]:
#anonimizar
#quitar descripción y tres decimales a las coordenadas
df_5 = df_4.copy()
df_5.drop(['descripcion'], axis=1, inplace=True)
df_5['latitud'] = df_5['latitud'].round(3)
df_5['longitud'] = df_5['longitud'].round(3)
df_5.head()

,precio,ambientes,no_baños,terreno_m2,año_constr,no_dormitorios,area_constr_m2,estacionamientos,latitud,longitud,tipo_de_propiedad,ofertado_como,ciudad,zona,provincia,dpto
0,67000.0,8.0,2.0,320.0,2017,3.0,140.0,NaN,-17.769,-63.066,Casa,En Venta,Santa Cruz de la Sierra,Este,Andrés Ibáñez,Santa Cruz
1,67000.0,5.0,2.0,91.0,2018,2.0,91.0,NaN,-17.804,-63.157,Departamento,En Venta,Santa Cruz de la Sierra,Sur,Andrés Ibáñez,Santa Cruz
2,68000.0,NaN,2.0,193.0,2015,3.0,87.0,NaN,-17.836,-63.245,Casa,En Venta,Santa Cruz de la Sierra,Suroeste,Andrés Ibáñez,Santa Cruz
3,68000.0,2.0,1.0,None,2016,1.0,62.83,NaN,-17.755,-63.196,Departamento,En Venta,Santa Cruz de la Sierra,Equipetrol/NorOeste,Andrés Ibáñez,Santa Cruz
4,68000.0,8.0,2.0,300.0,2022,3.0,110.0,NaN,-17.653,-63.173,Casa,En Venta,Santa Cruz de la Sierra,Norte,Andrés Ibáñez,Santa Cruz


In [43]:
#exportar a csv
#real_state_data_path = os.path.join(os.getcwd(), '../data/real_state_data_scz.csv')
real_state_data_path = '../data/real_state_data_after_cleaning.csv'
df_5.to_csv(real_state_data_path, index=False)